# Voxelized Phantom and Radiograph Simulation

This notebook demonstrates how to:
1. Download and load a high-resolution voxelized chest phantom from the XCIST repository.
2. Run a standard CT simulation using `VITools`.
3. Run a custom "Flat Panel" simulation (e.g., for a projected radiograph) using custom configuration files.

In [ ]:
import os
import requests
import zipfile
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import gecatsim as xc
from VITools.phantom import Phantom
from VITools.scanner import Scanner

## 1. Download and Prepare the Phantom

We will download the `Adult_Female_50percentile_Chest_Phantom` from the XCIST voxelized phantoms repository. The file is a zipped binary float32 array.

In [ ]:
def download_phantom(url, extract_to='.'):
    filename = url.split('/')[-1]
    if os.path.exists(filename):
        print(f"{filename} already exists. Skipping download.")
    else:
        print(f"Downloading {filename}...")
        # Use the raw URL for downloading
        raw_url = url.replace('blob', 'raw')
        response = requests.get(raw_url, stream=True)
        with open(filename, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        print("Download complete.")

    if not os.path.exists(extract_to):
        os.makedirs(extract_to)

    print(f"Extracting {filename}...")
    with zipfile.ZipFile(filename, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    print("Extraction complete.")
    
    # Return the path to the extracted bin file (assuming only one file in zip or we know the name)
    # Based on the zip name, we expect a file without extension or .bin inside
    base_name = filename.replace('.zip', '')
    extracted_files = os.listdir(extract_to)
    # Look for the large binary file
    for f in extracted_files:
        if base_name in f and not f.endswith('.zip'):
             return os.path.join(extract_to, f)
    return os.path.join(extract_to, base_name) # Fallback

# URL for the phantom
phantom_url = "https://github.com/xcist/phantoms-voxelized/blob/main/Adult_Female_50percentile_Chest_Phantom_1650x1050x880.zip"
phantom_path = download_phantom(phantom_url, extract_to='phantoms')

In [ ]:
# Load the phantom data
# Dimensions: 1650 (z) x 1050 (y) x 880 (x) based on filename
# Data type: float32
shape = (1650, 1050, 880) 
dtype = np.float32

print(f"Loading phantom from {phantom_path}...")
phantom_data = np.fromfile(phantom_path, dtype=dtype)
if phantom_data.size != np.prod(shape):
    print(f"Warning: Size mismatch. Expected {np.prod(shape)}, got {phantom_data.size}. Check dimensions.")
else:
    phantom_data = phantom_data.reshape(shape)
    print(f"Phantom loaded successfully with shape {phantom_data.shape}.")

# Create the VITools Phantom object
# We define an isotropic voxel size. For a chest phantom of this resolution,
# 0.5mm is a reasonable guess for a 50th percentile adult female (approx 44cm width).
voxel_spacing = (0.5, 0.5, 0.5) # (dz, dx, dy)
phantom = Phantom(phantom_data, spacings=voxel_spacing, patient_name='Adult_Female_50p', patientid=123)
print(phantom)

## 2. Standard CT Simulation

First, we initialize a default scanner and run a standard helical or axial CT scan.

In [ ]:
# Initialize Scanner
scanner_ct = Scanner(phantom, scanner_model="Scanner_Default", output_dir="output_ct")

# Visualize the setup (Scout View)
plt.figure(figsize=(10, 4))
scanner_ct.scout_view()
plt.show()

In [ ]:
# Run Scan and Reconstruction
# We'll scan a smaller range to save time for this example, e.g., 50mm centered.
center_z = 0
scan_range = 20
start_z = center_z - scan_range / 2
end_z = center_z + scan_range / 2

print("Running CT Scan...")
scanner_ct.run_scan(mA=50, kVp=120, startZ=start_z, endZ=end_z, pitch=1.0)

print("Running Reconstruction...")
scanner_ct.run_recon(fov=500, slice_thickness=1.0)

# Visualize a reconstructed slice
if scanner_ct.recon is not None and len(scanner_ct.recon) > 0:
    mid_slice = scanner_ct.recon.shape[0] // 2
    plt.imshow(scanner_ct.recon[mid_slice], cmap='gray', vmin=-1000, vmax=1000)
    plt.title(f"Reconstructed Slice {mid_slice}")
    plt.colorbar(label='HU')
    plt.show()

# Save DICOM
dicom_path = Path("output_ct") / "dicom" / "ct_scan.dcm"
scanner_ct.write_to_dicom(dicom_path)
print(f"DICOM saved to {dicom_path.parent}")

## 3. Flat Panel / Radiograph Simulation

Now we demonstrate a custom use case using specific configuration files for a Flat Panel detector. This could be used to simulate a single projected radiograph or a specific acquisition protocol.

In [ ]:
# Create the configuration files directory
cfg_dir = Path('cfg')
cfg_dir.mkdir(exist_ok=True)

# Define config content
protocol_cfg = """
protocol.scanTypes = [1, 1, 1]              # flags for airscan, offset scan, phantom scan 
 
# Table and gantry 
protocol.scanTrajectory = "Gantry_Helical"	# name of the function that defines the scanning trajectory and model 
protocol.viewsPerRotation = 360	    	    # total numbers of view per rotation 
protocol.viewCount = 360                    # total number of views in scan 
protocol.startViewId = 0		            # index of the first view in the scan 
protocol.stopViewId = protocol.startViewId+protocol.viewCount-1 # index of the last view in the scan 
protocol.airViewCount = 1		            # number of views averaged for air scan 
protocol.offsetViewCount = 1		        # number of views averaged for offset scan 
protocol.rotationTime = 1.0		            # gantry rotation period (in seconds) 
protocol.rotationDirection = 1		        # gantry rotation direction (1=CW, -1 CCW, seen from table foot-end) 
protocol.startAngle = 0		                # relative to vertical y-axis (n degrees) 
protocol.tableSpeed = 0		                # speed of table translation along positive z-axis (in mm/sec) 
protocol.startZ = 0		                    # start z-position of table 
protocol.tiltAngle = 0		                # gantry tilt angle towards negative z-axis (in degrees) 
protocol.wobbleDistance = 0.0		        # focalspot wobble distance 
protocol.focalspotOffset = [0, 0, 0]		# focalspot position offset 
 
# X-ray tube technique and filtration 
protocol.mA = 20		                    # tube current (in mA) 
protocol.spectrumCallback = "Spectrum"		# name of function that reads and models the X-ray spectrum 
protocol.spectrumFilename = "tungsten_tar7_120_unfilt.dat" # name of the spectrum file 
protocol.spectrumScaling = 1		        # scaling factor such that spectrum is in photons / mA / s / mm^2 at 1000 mm 
protocol.bowtie = ""       # name of the bowtie file (or []) 
protocol.filterCallback = "Xray_Filter"     # name of function to compute additional filtration 
protocol.flatFilter = ['al',3.0]            # additional filtration - materials and thicknesses (in mm) 
protocol.dutyRatio = 1.0                    # tube ON time fraction (for pulsed tubes) 
"""

physics_cfg = """
# Geometric and energy sampling 
physics.energyCount = 20 
physics.monochromatic = -1 
physics.colSampleCount = 1 
physics.rowSampleCount = 1 
physics.srcXSampleCount = 1 
physics.srcYSampleCount = 1 
physics.viewSampleCount = 1 
 
# Pre-image sampling and ways to model detector aliasing for flat-panel detector 
physics.FlatPanel_OSfactor = 1 # from 3, when  > 1 seems to break prepview 
physics.DetectorAliasMode = 1 
 
# Flags to determine what has to be recalculated each view 
physics.recalcDet = 0 
physics.recalcSrc = 0 
physics.recalcRayAngle = 0 
physics.recalcSpec = 0 
physics.recalcFilt = 0 
physics.recalcFlux = 0 
physics.recalcPht = 0 
physics.recalcDet = 0 
 
# Noise on/off settings 
physics.enableQuantumNoise = 0 
physics.enableElectronicNoise = 0 
 
# Internal physics models 
physics.rayAngleCallback = "Detector_RayAngles_2D" 
physics.fluxCallback = "Detection_Flux" 
physics.scatterCallback = "" 
physics.prefilterCallback = "Detection_prefilter" 
physics.crosstalkCallback = "" 
physics.lagCallback = "" 
physics.opticalCrosstalkCallback = "Detection_OpticalCrosstalk_FlatPanel" 
physics.DASCallback = "Detection_DAS_FlatPanel" 
 
# I/O preferences 
physics.outputCallback = "WriteRawView" 
"""

scanner_cfg = """
# Scanner geometry 
scanner.detectorCallback = "Detector_FlatPanel" # name of function that defines the detector shape and model 
scanner.sid = 540.0                         # source-to-iso distance (in mm) 
scanner.sdd = 950.0                         # source-to-detector distance (in mm) 
scanner.detectorColCount = 500             # total number of detector columns 
scanner.detectorRowCount = 500             # total number of detector rows 
scanner.detectorColOffset = 0.0             # detector column offset relative to centered position (in detector columns) 
scanner.detectorRowOffset = 0.0             # detector row offset relative to centered position (in detector rows) 
scanner.detectorColSize = 0.5               # detector column pitch or size (in mm) 
scanner.detectorRowSize = 0.5               # detector row pitch or size (in mm) 
scanner.detectorPrefilter = ['graphite', 1.0]  # detector filter 
 
# X-ray tube 
scanner.focalspotCallback = "SetFocalspot"  # name of function that defines the focal spot shape and model 
scanner.focalspotShape = "Uniform"          # Parameterize the model 
scanner.targetAngle = 7.0                   # target angle relative to scanner XY-plane (in degrees) 
scanner.focalspotWidth = 1.0 
scanner.focalspotLength = 1.0 
 
# Detector 
scanner.detectorMaterial = "lumex"          # detector sensor material 
scanner.detectorDepth = 3.0                 # detector sensor depth (in mm) 
scanner.detectionCallback = "Detection_EI"  # name of function that defines the detection process (conversion from X-rays to detector signal) 
scanner.detectionGain = 15.0                # factor to convert energy to electrons (electrons / keV) 
scanner.FlatPanel_H = 0.2026                # Parameter controlling the spread of OTF (flat-panel detector) 
scanner.detectorColFillFraction = 0.8       # active fraction of each detector cell in the column direction 
scanner.detectorRowFillFraction = 0.8       # active fraction of each detector cell in the row direction 
scanner.eNoise = 0.0                     # standard deviation of Gaussian electronic noise (in electrons)
"""

# Write configs to file
with open(cfg_dir / 'protocol_flat_panel.cfg', 'w') as f:
    f.write(protocol_cfg)
with open(cfg_dir / 'physics_flat_panel.cfg', 'w') as f:
    f.write(physics_cfg)
with open(cfg_dir / 'scanner_flat_panel.cfg', 'w') as f:
    f.write(scanner_cfg)
    
print("Config files created in cfg/")

In [ ]:
# Initialize a new scanner object for the flat panel simulation
scanner_fp = Scanner(phantom, output_dir="output_flat_panel")

# Load the custom configurations
print("Loading custom configuration...")
scanner_fp.xcist.load_cfg(
    cfg_dir / 'protocol_flat_panel.cfg', 
    cfg_dir / 'physics_flat_panel.cfg', 
    cfg_dir / 'scanner_flat_panel.cfg'
)

# Note: The Scanner.run_scan() wrapper overrides many protocol parameters (mA, views, etc.)
# to simplify standard use. For this custom config, we interact with the underlying 
# XCIST object directly or use run_scan with care. Here we run directly to respect
# the loaded config exactly.

# Prepare output directory manually since we are bypassing run_scan
results_dir = scanner_fp.output_dir / 'simulations' / f'{phantom.patientid}_flat_panel'
results_dir.mkdir(exist_ok=True, parents=True)
scanner_fp.xcist.cfg.experimentDirectory = str(results_dir)
scanner_fp.xcist.cfg.resultsName = str((results_dir / 'flat_panel_scan').absolute())

# Ensure the protocol points to the correct spectrum file path if it's not absolute
# The config uses "tungsten_tar7_120_unfilt.dat". We need to ensure XCIST finds it.
# XCIST usually looks in its installed spectrum directory. We might need to copy it or point to it.
spectrum_file = scanner_fp.xcist.cfg.protocol.spectrumFilename
if not os.path.isabs(spectrum_file):
    # Try to find it in XCIST installation
    xc_spectrum_path = Path(xc.__file__).parent / 'spectrum' / spectrum_file
    if xc_spectrum_path.exists():
        scanner_fp.xcist.cfg.protocol.spectrumFilename = str(xc_spectrum_path)
    else:
        print(f"Warning: Spectrum file {spectrum_file} not found in defaults. Sim may fail.")

# Run the simulation
print("Running Flat Panel Simulation...")
scanner_fp.xcist.run_all()

In [ ]:
# Visualize the results
# The physics.outputCallback is "WriteRawView", which outputs raw floats.
# Output filename pattern is usually resultsName.raw? Or resultsName.0000 etc.
# With WriteRawView, XCIST typically appends view indices or creates a large file.
# Let's check what was created.

import glob
output_files = sorted(glob.glob(f"{scanner_fp.xcist.cfg.resultsName}*"))
print(f"Found output files: {output_files}")

# Assuming a single large raw file or individual view files.
# Based on standard XCIST WriteRawView behavior, it might write .scan or similar, 
# or just append to the resultsName file if no extension was given in the C code, 
# but usually it adds .view0000 etc or just raw data.
# Let's try to read the main file if it exists, or the first found.

if output_files:
    # Pick the likely data file (usually the one without extensions like .cfg or .log if any)
    data_file = output_files[0]
    # Read raw data
    # Dimensions: viewCount x rows x cols
    cols = scanner_fp.xcist.cfg.scanner.detectorColCount
    rows = scanner_fp.xcist.cfg.scanner.detectorRowCount
    views = scanner_fp.xcist.cfg.protocol.viewCount
    
    print(f"Reading data from {data_file} with shape ({views}, {rows}, {cols})...")
    try:
        raw_data = np.fromfile(data_file, dtype=np.float32)
        # Check size
        expected_size = views * rows * cols
        if raw_data.size >= rows * cols: # At least one view
             # Reshape to 3D if full size, or 2D if partial/single view
             if raw_data.size == expected_size:
                 raw_data = raw_data.reshape((views, rows, cols))
             elif raw_data.size % (rows * cols) == 0:
                 actual_views = raw_data.size // (rows * cols)
                 raw_data = raw_data.reshape((actual_views, rows, cols))
             
             # Display the first view (projection)
             plt.figure(figsize=(6, 6))
             plt.imshow(raw_data[0] if raw_data.ndim == 3 else raw_data, cmap='gray')
             plt.title("First Projection View (Flat Panel)")
             plt.colorbar()
             plt.show()
             
             # Create a "Recon" object just to use the write_to_dicom method to save this projection
             # Note: This abuses the recon slot to save projection data as a DICOM image.
             scanner_fp.recon = raw_data if raw_data.ndim == 3 else raw_data[None]
             # Update config to match dimensions for DICOM writer (fov/imageSize)
             scanner_fp.xcist.cfg.recon.fov = 400 # arbitrary for projection
             scanner_fp.xcist.cfg.recon.imageSize = cols
             scanner_fp.xcist.cfg.recon.sliceThickness = 1.0
             
             dicom_fp_path = Path("output_flat_panel") / "dicom" / "flat_panel_proj.dcm"
             scanner_fp.write_to_dicom(dicom_fp_path)
             print(f"Projection saved to {dicom_fp_path.parent}")
             
        else:
            print(f"Data size {raw_data.size} does not match expected dimensions.")
            
    except Exception as e:
        print(f"Error reading or visualizing data: {e}")
else:
    print("No output files found.")